In [37]:
decode(x::Float16) = (b=bitstring(x); (b[1], b[2:6], b[7:16]))
decode(x::Float32) = (b=bitstring(x); (b[1], b[2:9], b[10:32]))
decode(x::Float64) = (b=bitstring(x); (b[1], b[2:12], b[13:64]))

decode (generic function with 3 methods)

In [42]:
[decode(Float16(1//3)), decode(Float32(1//3)), decode(Float64(1//3)), decode(Float64(Float16(1//3)))]

4-element Vector{Tuple{Char, String, String}}:
 ('0', "01101", "0101010101")
 ('0', "01111101", "01010101010101010101011")
 ('0', "01111111101", "0101010101010101010101010101010101010101010101010101")
 ('0', "01111111101", "0101010101000000000000000000000000000000000000000000")

In [46]:
c = Float32[]
n = Float32(1)
while n < 1000000
    push!(c, n)
    n = nextfloat(n)
end
# map(x-> bitstring(x),c)